# import

In [95]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline


In [96]:
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   



# read exemple

In [97]:
dept = '38'

td007 =pd.read_csv(data_dir/dept/'td007_paroi_opaque.csv',dtype=str)
td006 =pd.read_csv(data_dir/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/dept/'td001_dpe.csv',dtype=str)
td008 =pd.read_csv(data_dir/dept/'td008_baie.csv',dtype=str)


In [98]:
from assets_orm import DPEMetaData

In [99]:
meta = DPEMetaData()
self=meta

In [100]:
table =td008.copy()

In [101]:
table = meta.merge_all_tr_table(table)

table = meta.merge_all_tv_table(table)

In [102]:
table['tv010_coefficient_transmission_thermique_baie_id']

0         TV010_201
1         TV010_201
2         TV010_201
3         TV010_228
4         TV010_227
            ...    
527281    TV010_391
527282    TV010_337
527283    TV010_337
527284    TV010_391
527285    TV010_337
Name: tv010_coefficient_transmission_thermique_baie_id, Length: 527286, dtype: category
Categories (335, object): [TV010_001, TV010_002, TV010_003, TV010_004, ..., TV010_410, TV010_411, TV010_412, TV010_413]

In [103]:
td008_types = {'id': 'object',
               'td007_paroi_opaque_id': 'object',
               'reference': 'object',
               'td008_baie_id': 'object',
               'deperdition': 'float',
               'tv009_coefficient_transmission_thermique_vitrage_id': 'category',
               'presence_survitrage': 'bool',
               'coefficient_transmission_thermique_baie': 'float',
               'tv010_coefficient_transmission_thermique_baie_id': 'category',
               'tv011_resistance_additionnelle_id': 'category',
               'tv012_coef_transmission_thermique_baie_protection_solaire_id': 'category',
               'surface': 'float',
               'perimetre': 'float',
               'tv013_valeur_pont_thermique_id': 'category',
               'facteur_solaire': 'float',
               'tv021_facteur_solaire_id': 'category',
               'tv022_coefficient_masques_proches_id': 'category',
               'coefficient_masques_lointains_non_homogenes': 'float',
               'tv023_coefficient_masques_lointains_homogenes_id': 'category',
               'tv020_coefficient_orientation_id': 'category',
               'tv009_Type de vitrage': 'category',
               'tv009_Orientation': 'category',
               'tv009_Remplissage': 'category',
               'tv009_Epaisseur Lame': 'category',
               'tv009_Traitement du vitrage': 'category',
               'tv009_Ug': 'category',
               'tv010_Type de matériaux': 'category',
               'tv010_Type de Baie': 'category',
               'tv010_Ug': 'category',
               'tv010_Uw': 'category',
               'tv011_Fermetures': 'category',
               'tv011_∆R': 'category',
               'tv012_Uw': 'category',
               'tv012_∆R': 'category',
               'tv012_Ujn': 'category',
               'tv013_Type de liaison': 'category',
               'tv013_Isolation Mur': 'category',
               'tv013_Isolation Plancher bas': 'category',
               'tv013_Largeur du dormant': 'category',
               'tv013_Type de pose': 'category',
               "tv013_Retour d'isolation": 'category',
               'tv013_K': 'category',
               'tv021_Type de Pose': 'category',
               'tv021_Matériaux': 'category',
               'tv021_Type de Baie': 'category',
               'tv021_Type de Vitrage': 'category',
               'tv021_Fts': 'category',
               'tv022_Type de maque': 'category',
               'tv022_Avancé L': 'category',
               'tv022_Orientation': 'category',
               'tv022_Rapport L1/L2': 'category',
               'tv022_β & γ': 'category',
               'tv022_Fe1': 'category',
               'tv023_Hauteur α (°)': 'category',
               'tv023_Orientation': 'category',
               'tv023_Fe2': 'category',
               'tv020_Inclinaison de la paroi': 'category',
               'tv020_Orientation de la paroi': 'category',
               'tv020_Type de baie': 'category',
               'tv020_C1': 'category'}


In [104]:
table=table.astype(td008_types)

In [105]:
table['fen_lib_from_tv009'] = table['tv009_Type de vitrage'].astype('string') + ' ' + table['tv009_Remplissage'].astype('string').fillna('') + ' ' 
table['fen_lib_from_tv009'] +=table[
    'tv009_Epaisseur Lame'].astype('Int32').fillna(0).astype(int).astype(str).replace('0', '').apply(
    lambda x: x + ' mm ' if x != '' else x) + table['tv010_Type de matériaux'].astype('string').fillna('') + ' ' + table[
                              'tv009_Traitement du vitrage'].astype('string').fillna('')
table['fen_lib_from_tv009'] = table['fen_lib_from_tv009'].fillna('NONDEF')

table['fen_lib_from_tv021'] = table['tv021_Type de Baie'].astype('string') + ' ' + table['tv021_Type de Vitrage'].astype('string').fillna('') + ' ' 
table['fen_lib_from_tv021'] += table['tv021_Matériaux'].astype('string').fillna('') 
table['fen_lib_from_tv021'] = table['fen_lib_from_tv021'].fillna('NONDEF')



In [106]:
double_vitrage = table.fen_lib_from_tv009.str.lower().str.contains('double')|table.fen_lib_from_tv021.str.lower().str.contains('double')

triple_vitrage = table.fen_lib_from_tv009.str.lower().str.contains('triple')|table.fen_lib_from_tv021.str.lower().str.contains('triple')

simple_vitrage = table.fen_lib_from_tv009.str.lower().str.contains('simple')|table.fen_lib_from_tv021.str.lower().str.contains('simple')

In [107]:
table['type_vitrage_simple_infer']='NONDEF'

table.loc[double_vitrage,'type_vitrage_simple_infer']='double vitrage'
table.loc[triple_vitrage,'type_vitrage_simple_infer']='triple vitrage'
table.loc[simple_vitrage,'type_vitrage_simple_infer']='simple vitrage'

table.loc[simple_vitrage&double_vitrage,'type_vitrage_simple_infer'] = "INCOHERENT"
table.loc[simple_vitrage&triple_vitrage,'type_vitrage_simple_infer'] = "INCOHERENT"
table.loc[triple_vitrage&double_vitrage,'type_vitrage_simple_infer'] = "INCOHERENT"


In [112]:
table['type_vitrage_simple_infer'].value_counts()

double vitrage    451452
NONDEF             37219
simple vitrage     28308
INCOHERENT          8819
triple vitrage      1488
Name: type_vitrage_simple_infer, dtype: int64

In [193]:
import pandas as pd

In [192]:
table.loc[table.type_vitrage_simple_infer=='INCOHERENT',['fen_lib_from_tv021','fen_lib_from_tv009']].to_dict(orient='index')

{'5473': {'fen_lib_from_tv021': 'Fenêtre battante ou coulissante Double vitrage Bois',
  'fen_lib_from_tv009': 'Simple vitrage  Bois ou bois/métal '},
 '5474': {'fen_lib_from_tv021': 'Fenêtre battante ou coulissante Double vitrage Bois',
  'fen_lib_from_tv009': 'Simple vitrage  Bois ou bois/métal '},
 '5475': {'fen_lib_from_tv021': 'Fenêtre battante ou coulissante Double vitrage Bois',
  'fen_lib_from_tv009': 'Simple vitrage  Bois ou bois/métal '},
 '5476': {'fen_lib_from_tv021': 'Fenêtre battante ou coulissante Double vitrage Bois',
  'fen_lib_from_tv009': 'Simple vitrage  Bois ou bois/métal '},
 '5477': {'fen_lib_from_tv021': 'Fenêtre battante ou coulissante Double vitrage Bois',
  'fen_lib_from_tv009': 'Simple vitrage  Bois ou bois/métal '},
 '5478': {'fen_lib_from_tv021': 'Fenêtre battante ou coulissante Double vitrage Bois',
  'fen_lib_from_tv009': 'Simple vitrage  Bois ou bois/métal '},
 '5479': {'fen_lib_from_tv021': 'Fenêtre battante ou coulissante Double vitrage Bois',
  'fen_

In [188]:
table.type_vitrage_simple_infer.value_counts()

double vitrage    451452
NONDEF             37219
simple vitrage     28308
INCOHERENT          8819
triple vitrage      1488
Name: type_vitrage_simple_infer, dtype: int64

In [167]:
table.count().sort_values(ascending=False).to_dict()

{'fen_lib_from_tv009': 527286,
 'tv020_coefficient_orientation_id': 527286,
 'td007_paroi_opaque_id': 527286,
 'reference': 527286,
 'deperdition': 527286,
 'presence_survitrage': 527286,
 'coefficient_transmission_thermique_baie': 527286,
 'surface': 527286,
 'perimetre': 527286,
 'tv013_valeur_pont_thermique_id': 527286,
 'facteur_solaire': 527286,
 'id': 527286,
 'tv013_Isolation Mur': 519648,
 'tv013_K': 519648,
 'tv013_Type de liaison': 519648,
 'tv013_Largeur du dormant': 510378,
 'tv013_Type de pose': 510378,
 'coefficient_masques_lointains_non_homogenes': 509884,
 'tv010_coefficient_transmission_thermique_baie_id': 486742,
 'tv010_Type de matériaux': 479318,
 'tv010_Type de Baie': 479318,
 'tv010_Uw': 479318,
 'tv020_Inclinaison de la paroi': 477292,
 'tv020_C1': 477292,
 'tv021_Matériaux': 475713,
 'tv021_Type de Pose': 475713,
 'tv021_facteur_solaire_id': 475713,
 'tv021_Fts': 475713,
 'tv021_Type de Baie': 474634,
 'tv021_Type de Vitrage': 474634,
 'tv020_Orientation de la p

In [81]:
table.columns

NameError: name 'table' is not defined

In [168]:
table.loc[nondef,['tv020_Inclinaison de la paroi','tv020_coefficient_orientation_id']]


,tv020_Inclinaison de la paroi,tv020_coefficient_orientation_id
86,NaN,TV020_014
94,NaN,TV020_014
102,NaN,TV020_014
112,NaN,TV020_014
114,Paroi Horizontale,TV020_013
...,...,...
527248,NaN,TV020_014
527249,NaN,TV020_014
527250,NaN,TV020_014
527251,NaN,TV020_014


In [160]:
table.count().to_dict()

{'id': 527286,
 'td007_paroi_opaque_id': 527286,
 'reference': 527286,
 'td008_baie_id': 713,
 'deperdition': 527286,
 'tv009_coefficient_transmission_thermique_vitrage_id': 294413,
 'presence_survitrage': 527286,
 'coefficient_transmission_thermique_baie': 527286,
 'tv010_coefficient_transmission_thermique_baie_id': 486742,
 'tv011_resistance_additionnelle_id': 389495,
 'tv012_coef_transmission_thermique_baie_protection_solaire_id': 162115,
 'surface': 527286,
 'perimetre': 527286,
 'tv013_valeur_pont_thermique_id': 527286,
 'facteur_solaire': 527286,
 'tv021_facteur_solaire_id': 475713,
 'tv022_coefficient_masques_proches_id': 445700,
 'coefficient_masques_lointains_non_homogenes': 509884,
 'tv023_coefficient_masques_lointains_homogenes_id': 279628,
 'tv020_coefficient_orientation_id': 527286,
 'tv009_Type de vitrage': 294413,
 'tv009_Orientation': 275529,
 'tv009_Remplissage': 275529,
 'tv009_Epaisseur Lame': 275529,
 'tv009_Traitement du vitrage': 275529,
 'tv009_Ug': 294413,
 'tv0

In [158]:
table.columns

Index(['id', 'td007_paroi_opaque_id', 'reference', 'td008_baie_id',
       'deperdition', 'tv009_coefficient_transmission_thermique_vitrage_id',
       'presence_survitrage', 'coefficient_transmission_thermique_baie',
       'tv010_coefficient_transmission_thermique_baie_id',
       'tv011_resistance_additionnelle_id',
       'tv012_coef_transmission_thermique_baie_protection_solaire_id',
       'surface', 'perimetre', 'tv013_valeur_pont_thermique_id',
       'facteur_solaire', 'tv021_facteur_solaire_id',
       'tv022_coefficient_masques_proches_id',
       'coefficient_masques_lointains_non_homogenes',
       'tv023_coefficient_masques_lointains_homogenes_id',
       'tv020_coefficient_orientation_id', 'tv009_Type de vitrage',
       'tv009_Orientation', 'tv009_Remplissage', 'tv009_Epaisseur Lame',
       'tv009_Traitement du vitrage', 'tv009_Ug', 'tv010_Type de matériaux',
       'tv010_Type de Baie', 'tv010_Ug', 'tv010_Uw', 'tv011_Fermetures',
       'tv011_∆R', 'tv012_Uw', 'tv012_

In [140]:
table.loc[nondef,['reference','tv009_coefficient_transmission_thermique_vitrage_id','coefficient_transmission_thermique_baie']]

,reference,tv009_coefficient_transmission_thermique_vitrage_id,coefficient_transmission_thermique_baie
0,Fenêtres Est,NaN,2.2
1,Fenêtres Est,NaN,2.6
2,Fenêtres Ouest,NaN,2.2
3,Fenêtres Ouest,NaN,2.2
4,Portes-fenêtres Ouest,NaN,2.2
...,...,...,...
527281,Portes-fenêtres Sud,NaN,2.4
527282,Fenêtres Sud,NaN,2.4
527283,Fenêtres Sud,NaN,2.4
527284,Portes-fenêtres Est,NaN,2.5


In [61]:
much_window=nb_window>5

In [66]:
nb_window=table.deperdition/(table.coefficient_transmission_thermique_baie*table.surface)

nb_window[table.surface==0]=np.nan
nb_window[table.coefficient_transmission_thermique_baie==0]=np.nan

table['nb_window_calc']=nb_window.round(0)

In [68]:
table['nb_window_calc']

0         1.0
1         1.0
2         1.0
3         1.0
4         1.0
         ... 
527281    1.0
527282    1.0
527283    1.0
527284    1.0
527285    1.0
Name: nb_window_calc, Length: 527286, dtype: float64

In [73]:
table.deperdition ==0

0         False
1         False
2         False
3         False
4         False
          ...  
527281    False
527282    False
527283    False
527284    False
527285    False
Name: deperdition, Length: 527286, dtype: bool

In [70]:
table.loc[much_window][['nb_window_calc','surface','deperdition','coefficient_transmission_thermique_baie']]

,nb_window_calc,surface,deperdition,coefficient_transmission_thermique_baie
837,6.0,0.16,3.00,2.9
2192,10.0,0.10,2.60,2.6
2418,9.0,2.60,36.60,1.5
2842,6.0,2.78,26.17,1.5
5448,6.0,3.40,26.52,1.4
...,...,...,...,...
501660,9.0,1.56,31.14,2.2
501661,12.0,1.56,40.48,2.2
501668,8.0,2.40,43.14,2.2
501670,5.0,1.56,18.75,2.2


In [50]:
nb_window

0         1.000320
1         1.000000
2         1.000000
3         1.000000
4         1.000000
            ...   
527281    0.999568
527282    1.000906
527283    1.000882
527284    1.000000
527285    1.003831
Length: 527286, dtype: float64

In [37]:
table.fen_lib_from_tv009=table.fen_lib_from_tv009.fillna('NONDEF')

In [41]:
table.head()

,id,td007_paroi_opaque_id,reference,td008_baie_id,deperdition,tv009_coefficient_transmission_thermique_vitrage_id,presence_survitrage,coefficient_transmission_thermique_baie,tv010_coefficient_transmission_thermique_baie_id,tv011_resistance_additionnelle_id,...,tv022_Fe1,tv023_Hauteur α (°),tv023_Orientation,tv023_Fe2,tv020_Inclinaison de la paroi,tv020_Orientation de la paroi,tv020_Type de baie,tv020_C1,orientation_infer,fen_lib_from_tv009
0,1159,1048,Fenêtres Est,NaN,6.25,NaN,True,2.2,TV010_201,TV011_005,...,1.0,Absence de masque,NaN,1.0,≥ 75°,Est,NaN,0.57,Est,NONDEF
1,1160,1048,Fenêtres Est,NaN,3.12,NaN,True,2.6,TV010_201,NaN,...,1.0,Absence de masque,NaN,1.0,≥ 75°,Est,NaN,0.57,Est,NONDEF
2,1161,1048,Fenêtres Ouest,NaN,9.68,NaN,True,2.2,TV010_201,TV011_005,...,1.0,Absence de masque,NaN,1.0,≥ 75°,Ouest,NaN,0.57,Ouest,NONDEF
3,1162,1048,Fenêtres Ouest,NaN,5.72,NaN,True,2.2,TV010_228,TV011_005,...,1.0,Absence de masque,NaN,1.0,≥ 75°,Ouest,NaN,0.57,Ouest,NONDEF
4,2084,1814,Portes-fenêtres Ouest,NaN,3.74,NaN,True,2.2,TV010_227,TV011_005,...,1.0,Absence de masque,NaN,1.0,≥ 75°,Ouest,NaN,0.57,Ouest,NONDEF


In [40]:
table.fen_lib_from_tv009.value_counts()/table.shape[0]

NONDEF                                                                                                       0.441645
Double vitrage Air Sec 6 mm Bois ou bois/métal Non Traités                                                   0.061987
Double vitrage Argon ou Krypton 16 mm PVC A isolation Thermique renforcé                                     0.049381
Double vitrage Air Sec 12 mm PVC Non Traités                                                                 0.041979
Double vitrage Air Sec 16 mm PVC Non Traités                                                                 0.041423
                                                                                                               ...   
Double vitrage Argon ou Krypton 15 mm Métal sans rupture de pont thermique A isolation Thermique renforcé    0.000002
Triple vitrage Argon ou Krypton 14 mm Métal sans rupture de pont thermique Non Traités                       0.000002
Double vitrage Air Sec 18 mm Portes simples en métal Non

In [32]:
table.query('fen_lib_from_tv009=="Double vitrage Argon ou Krypton 6 mm Portes simples en bois ou PVC A isolation Thermique renforcé"')

,id,td007_paroi_opaque_id,reference,td008_baie_id,deperdition,tv009_coefficient_transmission_thermique_vitrage_id,presence_survitrage,coefficient_transmission_thermique_baie,tv010_coefficient_transmission_thermique_baie_id,tv011_resistance_additionnelle_id,...,tv022_Fe1,tv023_Hauteur α (°),tv023_Orientation,tv023_Fe2,tv020_Inclinaison de la paroi,tv020_Orientation de la paroi,tv020_Type de baie,tv020_C1,orientation_infer,fen_lib_from_tv009
299010,15620191,12309958,Porte,NaN,6.0,TV009_029,True,3.5,TV010_001,TV011_005,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Inconnu,Double vitrage Argon ou Krypton 6 mm Portes si...
